In [1]:
#Libraries:
#Requests will allow you to send HTTP/1.1 requests using Python. With it, you can add content like headers, form data, multipart files, and parameters via simple Python libraries. It also allows you to access the response data of Python in the same way.
#BeautifulSoup


import requests
import bs4
from bs4 import BeautifulSoup
import pandas as pd
import time

URL = "https://www.indeed.com/jobs?q=data+scientist+%2420%2C000&l=Toronto&start=10"
#conducting a request of the stated URL above:
page = requests.get(URL)
#specifying a desired format of “page” using the html parser - this allows python to read the various components of the page, rather than treating it as one long string.
soup = BeautifulSoup(page.text, "html.parser")
#printing soup in a more structured tree format that makes for easier reading
#print(soup.prettify())

In [2]:
def extract_job_title_from_result(soup): 
    jobs = []
    for div in soup.find_all(name="div", attrs={"class":"row"}):
        for a in div.find_all(name="a", attrs={"data-tn-element":"jobTitle"}):
            jobs.append(a["title"])
    return(jobs)


extract_job_title_from_result(soup)

['Operations Research Analyst with DoD Secret or TS Clearance',
 'Predictive Modeling Actuary – Multiple Positions Available',
 'Data Scientist - Labor Planning & Analytics',
 'Operations Research Scientist',
 'Data Scientist',
 'Market Research Analyst - Technical',
 'Data Scientist (Relocation to Saudi Arabia)',
 'Sales Account Executive - Machine Learning – (Remote)',
 'Google Cloud Solutions Architect',
 'Pathology Surveyor',
 'Predictive Modeling Actuary, P&C',
 'Business Intelligence Engineer (f/m) (relocated to Germany)',
 'Geologist - Unconventional Resources (Relocation to Saudi Arabia)']

In [3]:
def extract_company_from_result(soup): 
    companies = []
    for div in soup.find_all(name="div", attrs={"class":"row"}):
        company = div.find_all(name="span", attrs={"class":"company"})
        if len(company) > 0:
            for b in company:
                companies.append(b.text.strip())
        else:
            sec_try = div.find_all(name="span", attrs={"class":"result-link-source"})
            for span in sec_try:
                companies.append(span.text.strip())
    return(companies)

extract_company_from_result(soup)

['DSoft Technology, Engineering and Analysis, Inc.',
 'DW Simpson Global Actuarial Recruitment',
 "DICK'S Sporting Goods",
 'Fedex Ground',
 'FedEx Services',
 "DICK'S Sporting Goods",
 'Saudi Aramco',
 'Cvedia PTE. LTD.',
 'Pythian',
 'The Joint Commission',
 'DW Simpson Global Actuarial Recruitment',
 'Avira Operations GmbH & Co. KG',
 'Saudi Aramco']

In [4]:
def extract_location_from_result(soup): 
    locations = []
    spans = soup.findAll('span', attrs={'class': 'location'})
    for span in spans:
        locations.append(span.text)
    return(locations)

extract_location_from_result(soup)

['Coraopolis, PA 15108',
 'Township of Moon, PA 15108',
 'Coraopolis, PA',
 'Coraopolis, PA 15108']

In [5]:
def extract_salary_from_result(soup): 
    salaries = []
    for div in soup.find_all(name="div", attrs={"class":"row"}):
        try:
            salaries.append(div.find('nobr').text)
        except:
            try:
                div_two = div.find(name="div", attrs={"class":"sjcl"})
                div_three = div_two.find("div")
                salaries.append(div_three.text.strip())
            except:
                salaries.append("Nothing_found")
    return(salaries)

extract_salary_from_result(soup)

['DSoft Technology, Engineering and Analysis, Inc.',
 'DW Simpson Global Actuarial Recruitment',
 'Nothing_found',
 'Nothing_found',
 'Nothing_found',
 'Nothing_found',
 'Saudi Aramco\n\n\n681 reviews',
 'Cvedia PTE. LTD.',
 'Pythian\n\n\n10 reviews',
 'The Joint Commission\n\n\n58 reviews',
 'DW Simpson Global Actuarial Recruitment',
 'Avira Operations GmbH & Co. KG',
 'Saudi Aramco\n\n\n681 reviews']

In [6]:
def extract_summary_from_result(soup): 
    summaries = []
    spans = soup.findAll('span', attrs={'class': 'summary'})
    for span in spans:
        summaries.append(span.text.strip())
    return(summaries)

extract_summary_from_result(soup)

['DSoft Technology is seeking Operations Research Analysts to support and assist the Headquarters Air Force Space Command with analysis of space systems...',
 'Well-known client seeks an ACAS or FCAS with 2+ years of multivariate predictive modeling experience, along with a high degree of proficiency with SAS and SQL.',
 'Contribute to the design and testing of new or enhanced staffing methodologies; evaluate the costs/benefits, present findings, and support the scaling of...',
 'Time management and organizational skills necessary to appropriately prioritize workload, plan for resources to meet deadlines and goals, and respond to...',
 'Responsible for descriptive, predictive, prescriptive and ensemble modeling, advanced statistical techniques, use of databases tools and/or other approaches...',
 'Manage ESRI’s ArcGIS Platform at Dick’s Sporting Goods including ArcGIS Desktop, ArcGIS online, ArcGIS Server and SQL server database by working with key...',
 'The Data Scientist primary rol

In [7]:
max_results_per_city = 10
city_set = ['New+York','Chicago','San+Francisco', 'Austin', 'Seattle', 'Los+Angeles', 'Philadelphia', 'Atlanta', 'Dallas','Pittsburgh', 'Portland', 'Phoenix', 'Denver', 'Houston', 'Miami', 'Washington+DC', 'Boulder']
#city_set = ['Toronto']
columns = ["city", "job_title", "company_name", "location", "summary", "salary"]
sample_df = pd.DataFrame(columns = columns)




In [8]:
for city in city_set:
    for start in range(0, max_results_per_city, 10):
        page = requests.get('http://www.indeed.com/jobs?q=data+scientist+%2420%2C000&l=' + str(city) + '&start=' + str(start))
        print("page: " + str(start))
        time.sleep(1)  #ensuring at least 1 second between page grabs
        soup = BeautifulSoup(page.text, "lxml", from_encoding="utf-8")
        for div in soup.find_all(name="div", attrs={"class":"row"}): 
            #specifying row num for index of job posting in dataframe
            num = (len(sample_df) + 1) 
            #creating an empty list to hold the data for each posting
            job_post = [] 
            #append city name
            job_post.append(city)
            #print(city)
            
            #grab job title
            for a in div.find_all(name="a", attrs={"data-tn-element":"jobTitle"}):
                job_post.append(a["title"])
                
            #grabbing company name
            company = div.find_all(name="span", attrs={"class":"company"}) 
            if len(company) > 0: 
                for b in company:
                    job_post.append(b.text.strip()) 
            else: 
                sec_try = div.find_all(name="span", attrs={"class":"result-link-source"})
                for span in sec_try:
                    job_post.append(span.text)
                    
            #grabbing location name
            c = div.findAll('span', attrs={'class': 'location'})
            if len(c) > 0:
                for span in c: 
                    job_post.append(span.text)
            else:
                    job_post.append("Nothing_found")
            #grabbing summary text
            d = div.findAll('span', attrs={'class': 'summary'}) 
            for span in d:
                job_post.append(span.text.strip()) 
            #grabbing salary
            try:
                job_post.append(div.find('nobr').text) 
            except:
                try:
                    div_two = div.find(name="div", attrs={"class":"sjcl"}) 
                    div_three = div_two.find("div") 
                    job_post.append(div_three.text.strip())
                except:
                    job_post.append("Nothing_found")         
                    
            print("job post:")
            print(job_post)
            print("job post length:")
            print(len(job_post))
            sample_df.loc[num] = job_post   
          
        #sample_df.loc[num] = job_post        
#return sample_df

page: 0


C:\Users\Larissa\Anaconda3\lib\site-packages\bs4\__init__.py:146: UserWarning: You provided Unicode markup but also provided a value for from_encoding. Your from_encoding will be ignored.
  warnings.warn("You provided Unicode markup but also provided a value for from_encoding. Your from_encoding will be ignored.")


job post:
['New+York', 'Data Scientist', 'Fora Financial LLC', 'Nothing_found', 'A Day in the Life of a Data Scientist:. Identify new data sources/patterns that add significant lift to predictive modeling capabilities....', 'Fora Financial LLC\n\n\n20 reviews']
job post length:
6
job post:
['New+York', 'Data Scientist', 'Liberty Lending', 'Nothing_found', 'Data scientist to work as a statistician, data miner, and business analyst for the Risk Department. Conduct data mining from internal and external data sources....', 'Liberty Lending']
job post length:
6
job post:
['New+York', 'Research Data Associate', 'NYU Langone Health', 'Nothing_found', 'Extracts data for publications, or provides data collection from outside physicians offices. Compiles data and assists in consolidating and analyzing data for...', 'NYU Langone Health\n\n\n752 reviews']
job post length:
6
job post:
['New+York', 'Quantitative Data Scientist', 'Harnham', 'Nothing_found', 'Quantatative Data Scientist. Their top pri

page: 0
job post:
['San+Francisco', 'Data Visualization Engineer', 'eBay Inc.', 'Nothing_found', 'Collaborate with data scientists, analysts, and data engineers to translate business questions into flexible tools and dashboards that scale across the...', 'eBay Inc.\n\n\n1,530 reviews']
job post length:
6
job post:
['San+Francisco', 'Senior Data Scientist', 'OpenTable', 'Nothing_found', 'Senior Data Scientist. 3+ years of industry experience as a Data Scientist or equivalent. Experience with data exploration and analysis....', 'OpenTable\n\n\n35 reviews']
job post length:
6
job post:
['San+Francisco', 'Data Scientist - Insurance', 'Blue Owl', 'Nothing_found', 'This role will require the candidate to be extremely hands on with building models and data driven tools. We invite you to help us build an InsurTech company...', 'Blue Owl']
job post length:
6
job post:
['San+Francisco', 'Associate Scientist /Scientist, DMPK', 'Genentech', 'Nothing_found', 'DMPK data analysis and modeling. Collab

page: 0
job post:
['Seattle', 'D3 - Data Science Intern (Bothell)', 'Allstate', 'Nothing_found', 'The Data Scientist Family requires Analytic Agility, the ability to quickly learn new modeling/machine learning techniques, programming languages, and see how...', 'Allstate\n\n\n5,668 reviews']
job post length:
6
job post:
['Seattle', 'Data Scientist', 'Indeed Prime', 'Nothing_found', "Indeed Prime is a free service that connects qualified job-seekers (that's you) with top companies hiring tech roles. With one application you can be...", 'Indeed Prime']
job post length:
6
job post:
['Seattle', 'Scientist, Hematopoietic Cell Differentiation', 'Astellas', 'Nothing_found', 'Accurately record procedures, data and results. Interpret data, draw conclusions from experimental work, and present results in team meetings....', 'Astellas\n\n\n152 reviews']
job post length:
6
job post:
['Seattle', 'Principal Data Scientist', 'Big Fish Games', 'Nothing_found', 'Mentor junior data analysts and data scie

...

page: 0
job post:
['Dallas', 'Data Scientist - Immediate need', 'MNR IT Solutions LLC', 'Nothing_found', 'Implements statistical data quality procedures on new data sources, and by applying rigorous iterative data analytics, supports Data Scientists in data sourcing...', 'MNR IT Solutions LLC']
job post length:
6
job post:
['Dallas', 'Machine Learning Engineer', 'Indeed Prime', 'Nothing_found', "Indeed Prime is a free service that connects qualified job-seekers (that's you) with top companies hiring tech roles. With one application you can be...", 'Indeed Prime']
job post length:
6
job post:
['Dallas', 'Senior Scientist/Scientist', 'Reata Pharmaceuticals, Inc.', 'Nothing_found', 'Reviewing, analyzing, and interpreting scientific data and results; Collaborating with other scientists and keeping current with field of expertise, including...', 'Reata Pharmaceuticals, Inc.']
job post length:
6
job post:
['Dallas', 'Machine Learning Engineer - Mid Level', 'USAA', 'Nothing_found', 'Purpose o

page: 0
job post:
['Portland', 'Data Scientist / Predictive Modeler', 'M Theory Solutions', 'Nothing_found', 'Ready to demonstrate your marketing analytics and data sciences prowess? You’ll have access to large B2B and B2C data sets on a robust analytic platform....', 'M Theory Solutions']
job post length:
6
job post:
['Portland', 'Machine Learning Engineer', 'Indeed Prime', 'Nothing_found', "Indeed Prime is a free service that connects qualified job-seekers (that's you) with top companies hiring tech roles. With one application you can be...", 'Indeed Prime']
job post length:
6
job post:
['Portland', 'Scientist, Immunohistochemistry and Histopathology Tissue Diagnostics', 'MolecularMD', 'Nothing_found', 'MolecularMD is expanding its Tissue Diagnostics group and seeks to hire a highly motivated, experienced Scientist to develop and validate tissue-based...', 'MolecularMD']
job post length:
6
job post:
['Portland', 'Data Scientist, Personalization', 'NIKE INC', 'Portland, OR', 'Experien

page: 0
job post:
['Denver', 'Data Scientist', 'Transamerica', 'Nothing_found', 'As part of the Advanced Analytics group, the Data Scientist works with product managers, data engineers, and other data scientists to develop innovative...', 'Transamerica\n\n\n621 reviews']
job post length:
6
job post:
['Denver', 'Machine Learning Engineer', 'Indeed Prime', 'Nothing_found', "Indeed Prime is a free service that connects qualified job-seekers (that's you) with top companies hiring tech roles. With one application you can be...", 'Indeed Prime']
job post length:
6
job post:
['Denver', 'Data Scientist', 'Indeed Prime', 'Nothing_found', "Indeed Prime is a free service that connects qualified job-seekers (that's you) with top companies hiring tech roles. With one application you can be...", 'Indeed Prime']
job post length:
6
job post:
['Denver', 'Research Scientist (Vision Technology)', 'Sensory, Inc', 'Nothing_found', 'Research Scientist (Vision Technology). As a Research Scientist within Sens

page: 0
job post:
['Miami', 'Data Scientist', 'TracFone Wireless', 'Nothing_found', 'The Data Scientist should be able to work with big data, experiment with various Machine Learning algorithms that will solve the task, create prototypes and...', 'TracFone Wireless\n\n\n91 reviews']
job post length:
6
job post:
['Miami', 'Data Science Instructor', 'Thinkful', 'Nothing_found', '3+ years experience as a data scientist, including Python experience. You’ll lead technical workshops teaching the basics of data science, primarily focusing on...', 'Thinkful']
job post length:
6
job post:
['Miami', 'Open Source Analyst', 'Leidos', 'Nothing_found', 'Experience with Boolean logic, natural language processing, social media analytics, data correlation, and data visualizations....', 'Leidos\n\n\n653 reviews']
job post length:
6
job post:
['Miami', 'Data Scientist- Predictive', 'Focus Global Talent Solutions', 'Nothing_found', 'Provided by Dice Data Scientist, Python, R, Machine Learning,. This Data 

page: 0
job post:
['Boulder', 'Data Scientist', 'Transamerica', 'Nothing_found', 'As part of the Advanced Analytics group, the Data Scientist works with product managers, data engineers, and other data scientists to develop innovative...', 'Transamerica\n\n\n621 reviews']
job post length:
6
job post:
['Boulder', 'Data Scientist / Broomfield, CO / $50hr / Contract-to-Hire', 'Vaco - Nashville', 'Nothing_found', 'Data scientists use data and analytical ability to find and interpret rich data sources; Merge data sources;...', 'Vaco - Nashville']
job post length:
6
job post:
['Boulder', 'Data Engineer', 'Transamerica', 'Nothing_found', 'Partner with data scientists, analytics engagement managers, and other data engineers to discover, collect, cleanse, and refine the data needed for analysis and...', 'Transamerica\n\n\n621 reviews']
job post length:
6
job post:
['Boulder', 'Machine Learning Engineer', 'Indeed Prime', 'Nothing_found', "Indeed Prime is a free service that connects qualified jo

In [11]:
sample_df

,city,job_title,company_name,location,summary,salary
1,New+York,Data Scientist,Fora Financial LLC,Nothing_found,A Day in the Life of a Data Scientist:. Identi...,Fora Financial LLC\n\n\n20 reviews
2,New+York,Data Scientist,Liberty Lending,Nothing_found,"Data scientist to work as a statistician, data...",Liberty Lending
3,New+York,Research Data Associate,NYU Langone Health,Nothing_found,"Extracts data for publications, or provides da...",NYU Langone Health\n\n\n752 reviews
4,New+York,Quantitative Data Scientist,Harnham,Nothing_found,Quantatative Data Scientist. Their top priorit...,Harnham\n\n\n6 reviews
5,New+York,AI Research Scientist,Bloomberg,Nothing_found,The infrastructure and algorithms we develop a...,Bloomberg\n\n\n670 reviews
6,New+York,Data Scientist,SupplyHouse.com,"Melville, NY",Discover new data points and validate existing...,Nothing_found
7,New+York,"Associate Data Scientist, Premium Analytics",Spotify,"New York, NY 10011 (Chelsea area)",You will work with a global team of world-clas...,Nothing_found
8,New+York,Data/NLP Scientist,Storm Global Analytics,"New York, NY",Design NLP models for searching structured/uns...,Nothing_found
9,New+York,"Data Scientist, Intern (2019)",1010data,"New York, NY",1010data is a highly differentiated product th...,Nothing_found
10,New+York,ASSOCIATE DATA SCIENTIST,"Foot Locker, Inc.","New York, NY",The Associate Data Scientist assists the team ...,Nothing_found
